# Исследование надежности заемщиков.

**Описание проекта**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - проверить следующие гипотезы:
1.  Существует зависимость между количеством детей и возвратом кредита в срок.
2.  Существует зависимость между семейным положением и возвратом кредита в срок.
3.  Существует зависимость между уровнем дохода и возвратом кредита в срок.
4.  Разные цели кредита влияют на его возврат в срок.

## Обзор данных

In [1]:
import pandas as pd # импорт библиотеки pandas

In [8]:
df = pd.read_csv('/datasets/data.csv') # читаем файл и сохраняем его в переменную df

In [3]:
df.head(10) # выведем на экран первые 10 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.info() # просмотрим основную информацию о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Согласно документации к данным:   
 
* `children` — количество детей в семье, тип данных
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита


In [5]:
df.describe() # просмотрим описательную статистику таблицы

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Выводы**

В каждой строке таблицы - информация о клиенте. Предварительно можно утверждать, что данных достаточно для проверки гипотез. Однако в данных были обнаружены следующие проблемы:
* отрицательные значения в столбцах 'children' и 'days_employed'
* пропуски в столбцах 'days_employed' и 'total_income'
* максимальное количество детей - 20, что возможно, но маловероятно
* минимальный возраст - 0
* столбы 'education' и 'family_status' излишни, так как их значения, по сути, дублируются в столбцах 'education_id' и 'family_status_id'
* в столбце 'education' постоянно меняется регистр

Чтобы двигаться дальше, необходимо устранить эти проблемы.

## Предобработка данных

### Заполнение пропусков

In [6]:
df.isna().sum() # подсчитаем суммарное количество пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Видим одинаковое количество пропусков в днях работы и в доходе. Доля пропусков составляет более 10%, что довольно много, удалять эти данные нельзя. Причины пропусков может быть две:
1. Эти люди никогда не работали официально или еще молоды (студенты);
2. Ошибки при сборе и/или выгрузке данных.

В завимости от причины действия с этими данными будут отличаться.
В большинстве случаев банки неохотно выдают кредиты тем, у кого нет официального дохода, однако есть специальные банки, которые выдают кредиты и в таких случаях. Проверим, наш ли этой случай.

In [7]:
df['income_type'].value_counts() # выведем все виды дохода

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Как мы видим, в списке встречаются студенты и безработные, однако их количество крайне невелико. Посмотрим, какой вид дохода у людей, у которых информация о доходе и днях работы отсутствует.

In [8]:
df[df['days_employed'].isna()].head(10) # выведем первые 10 строк с пропусками

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Беглого взгляда на данные достаточно, чтобы исключить первую причину: в графе "вид доходов" очень часто встречаются "пенсионер" и "госслужащий", которые явно имеют официальный доход. Следовательно, причина пропусков - ошбки при сборе и/или выгрузке данных. Заполним эти пропуски медианными значениями.

In [18]:
income_type_median = df.groupby('income_type')['total_income'].median().sort_values(ascending=False)
# расчитаем медиану для каждого источника дохода
income_type_median

income_type
предприниматель    499163.144947
компаньон          172357.950966
госслужащий        150447.935283
сотрудник          142594.396847
безработный        131339.751676
пенсионер          118514.486412
студент             98201.625314
в декрете           53829.130729
Name: total_income, dtype: float64

In [19]:
for i in income_type_median.keys(): # заполним пропуски медианным значением
    df.loc[df['income_type']==i, 'total_income'] = df.loc[df['income_type']==i, 'total_income'].fillna(income_type_median[i])

Аналогично для стажа работы:

In [10]:
days_employed_median = df.groupby('income_type')['days_employed'].median()
# расчитаем медиану для каждого источника дохода

for i in days_employed_median.keys(): # заполним пропуски медианным значением
    df.loc[df['income_type']==i, 'days_employed'] = df.loc[df['income_type']==i, 'days_employed'].fillna(days_employed_median[i])

In [12]:
df.isna().sum() # удостоверимся, что пропусков в данных не осталось

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Удаление дубликатов

При обзоре данных были обнаружены неявные дубликаты в столбце 'education'. Удалим их.

In [13]:
df['education'].unique() # посмотрим уникальные значения столбца 'education'

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [14]:
df['education'] = df['education'].str.lower() # приведем все значения к нижнему регистру

In [15]:
df['education'].unique() # проверим отсутствие неявных дублей

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Теперь удалим явные дубликаты.

In [16]:
df.duplicated().sum() # подсчет количества дубликатов

71

In [17]:
df = df.drop_duplicates().reset_index(drop = True) # удаляем дубликаты, сбрасывая индекс

In [18]:
df.duplicated().sum() # проверка на отсутствие дубликатов

0

### Проверка данных на аномалии и исправления

Рассмотрим каждый столбец в отдельности и поищем аномалии.

In [19]:
df['children'].unique() # проверим количество детей

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Видим две странности: значения -1 и 20. Посмотрим, сколько таких значений.

In [20]:
df[df['children'] == -1]['children'].count() # считаем количество значений -1

47

Это значение явно является ошибкой. Заменим -1 на 1.

In [21]:
df['children'] = df['children'].replace(-1, 1) # производим замену

In [22]:
df[df['children'] == -1]['children'].count() # проверим, сработала ли замена

0

С отрицательным значениями разобрались, приступим к владельцам 20 детей.

In [23]:
df[df['children'] == 20]['children'].count() # считаем количество значений 20

76

Значение немалое, однако, возможно, наш банк выдает какие-то крайне выгодные кредиты семьям с 20 детьми? Сгруппируем по возрасту и проверим.

In [24]:
df[df['children'] == 20].groupby('dob_years')['dob_years'].count()

dob_years
0     1
21    1
23    1
24    1
25    1
26    1
27    2
29    2
30    3
31    2
32    2
33    2
34    3
35    2
36    2
37    4
38    1
39    1
40    4
41    2
42    3
43    2
44    2
45    3
46    3
48    1
49    3
50    3
51    1
52    1
53    1
54    1
55    1
56    5
57    1
59    2
60    1
61    1
62    1
64    1
69    1
Name: dob_years, dtype: int64

Если бы значения были от 40 лет, можно было бы преположить, что наше предположение верно, однако в нашей ситуации считаем эти данные некорректными и заменим значение на 2.

In [25]:
df['children'] = df['children'].replace(20, 2) # производим замену

In [26]:
df['children'].unique() # проверим результат замен

array([1, 0, 3, 2, 4, 5])

С детьми разобрались, приступим к стажу работы.

In [27]:
df['days_employed'].describe() # проверим стаж работы

count     21454.000000
mean      63304.511872
std      140947.072148
min      -18388.949901
25%       -2575.665602
50%       -1354.642207
75%        -317.312251
max      401755.400475
Name: days_employed, dtype: float64

Сразу видим несколько проблем: 
1. Отрицательные значения;
2. Крайне высокое максимальное значение; 
3. Значения в вещественных числах, а не в целых.

Разберемся с отрицательными значениями.

In [28]:
df['days_employed'] = df['days_employed'].abs() # берем значения по модулю

In [29]:
df['days_employed'].describe() # убедимся, что замена значений сработала

count     21454.000000
mean      67059.260281
std      139199.752746
min          24.141633
25%        1023.662702
50%        1996.223132
75%        5320.665263
max      401755.400475
Name: days_employed, dtype: float64

От отрицательных значений избавились. С этими данными можно еще много поработать: разобраться в причине такого высокого максимального значения, перевести дни в месяцы или годы, так как с ними удобнее работать. Однако в данном ислледовании нет необходимости этим заниматься, так как задачи "Выяснить, есть ли зависимость между стажем работы и возвратом кредита в срок" не стоит, потому пока что оставим эти данные так, как есть.

In [30]:
sorted(df['dob_years'].unique()) # посмотрим, какие возраста представлены в выборке

[0,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75]

Значение возраста "0" явно выглядит как аномалия, заменим на медиану.

In [31]:
df['dob_years'] = df['dob_years'].replace(0,  int(df['dob_years'].median()))
#заменяем все значения, равные нулю, на медиану

In [32]:
sorted(df['dob_years'].unique()) # проверим возраста

[19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75]

Аномалии в значениях возраста обработаны.

In [33]:
df['family_status'].unique() # посмотрим значения семейного статуса

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

С семейным статусом проблем нет.

In [34]:
df['gender'].unique() # посмотрим значения пола

array(['F', 'M', 'XNA'], dtype=object)

Видим странное значение 'XNA'. Выведем строки, в которых указан этот пол.

In [35]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10684,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Такая строка только одна, и значения в остальных столбцах в норме, потому трогать ее не будем.

In [36]:
df['income_type'].unique() # посмотрим значения типа занятости

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

С типом занятости проблем нет.

In [37]:
df['debt'].unique() # посмотрим значения факта наличия задолженности

array([0, 1])

Cо значением факта долга наличия задолженности проблем нет.

In [38]:
df['total_income'].describe()

count    2.145400e+04
mean     1.653201e+05
std      9.818730e+04
min      2.066726e+04
25%      1.076239e+05
50%      1.425944e+05
75%      1.958209e+05
max      2.265604e+06
Name: total_income, dtype: float64

Значения доходов с 6 знаками после запятой в контексте задачи обладают избыточной точностью, которой мы можем принебречь ради удобства. Приведем значения к целочисленному типу.

In [39]:
df['total_income'] = df['total_income'].astype(int) # преобразуем тип float в int

In [40]:
df['purpose'].unique() # посмотрим цель взятие кредита

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Аномалий нет, однако в дальнейшем необходимо будет категоризировать эти данные.

In [41]:
df.head(10) # посмотрим таблицу после предобработки

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

При обзоре данных выяснилось, что в датафрейме присутствуют дублирующие признаки. Создадим специальные словари для таких признаков.

In [42]:
education_dict = df[['education', 'education_id']] # создаем словарь для образования
education_dict = education_dict.drop_duplicates().reset_index(drop = True) # удаляем дубликаты, сбрасываем индексы
education_dict # выводим словарь на экран

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [43]:
family_dict = df[['family_status', 'family_status_id']] # создаем словарь для семейного положения
family_dict = family_dict.drop_duplicates().reset_index(drop = True) # удаляем дубликаты, сбрасываем индексы
family_dict # выводим словарь на экран

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [44]:
df = df.drop(['education', 'family_status'], axis = 1) # удаляем лишние столбцы из датафрейма
df.head(10) # просмотрим таблицу после удаления столбцов

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Категоризация дохода.

Для проверки гипотез необходимо категоризировать доходы. Поделим доходы на 5 категорий:
* если доход до 30000 - E
* если доход до 50000 - D
* если доход до 200000 - C
* если доход до 1000000 - B
* если доход более 1000000 - A

In [45]:
def income_category(total_income):   
    if total_income <= 30000:
            return 'E'
    if total_income <= 50000:
            return 'D'
    if total_income <= 200000:
            return 'C'
    if total_income <= 1000000:
            return 'B'    
    return 'A'

In [46]:
df['total_income_category'] = df['total_income'].apply(income_category) 
# создаем столбец total_income_category, в который записываем категорию дохода

In [47]:
df['total_income_category'].value_counts() # проверим корректность категоризации

C    16015
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Категоризация целей кредита.

Помимо дохода, необходимо также категоризировать цели кредита.

In [48]:
def purpose_category(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'жи' in purpose:
        return 'операции с недвижимостью'
    if 'свад' in purpose:
        return 'проведение свадьбы'
    if 'образ' in purpose:
        return 'получение образования'
    return 'цель не определена'

In [49]:
df['purpose_category'] = df['purpose'].apply(purpose_category)
# создаем столбец purpose_category, в который записываем цель кредита

In [50]:
df['purpose_category'].value_counts() # проверим корректность категоризации

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

## Ответы на вопросы.

### Зависимость между наличием детей и возвратом кредита в срок

Посмотрим отношение наличия задолженности по возврату кредитов к общему количеству кредитов.

In [59]:
def create_debt_ratio_table(dependence): # объявляем функцию
    table = df.pivot_table(index=[dependence], values=['debt'],  aggfunc=['count', 'sum', 'mean']) # создаем сводную таблицу
    table.columns = ['total', 'debt', 'ratio']
    return table

In [60]:
create_debt_ratio_table('children')

,total,debt,ratio
children,,,
0,14091,1063,0.075438
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод:** как видим, те, у кого нет детей, возвращают кредиты чаще, чем те, у кого дети есть. Сделать корректный вывод о том, как часто возвращают долги те, у кого 5 детей, невозможно, так как выборка нерепрезентативна.

### Зависимость между семейным положением и возвратом кредита в срок

In [61]:
family_dict.merge(create_debt_ratio_table('family_status_id'), on='family_status_id', how='left').drop('family_status_id', axis=1)

,family_status,total,debt,ratio
0,женат / замужем,12339,931,0.075452
1,гражданский брак,4151,388,0.093471
2,вдовец / вдова,959,63,0.065693
3,в разводе,1195,85,0.071130
4,Не женат / не замужем,2810,274,0.097509


Как ни странно, однако вдовы/вдовцы, а также разведенные люди возвращают кредит в срок чаще всего. А вот те, кто не состоят в браке или состоят в гражданском, реже остальных возвращают кредит вовремя.

### Зависимость между уровнем дохода и возвратом кредита в срок

In [62]:
create_debt_ratio_table('total_income_category')

,total,debt,ratio
total_income_category,,,
A,25,2,0.080000
B,5042,356,0.070607
C,16015,1360,0.084920
D,350,21,0.060000
E,22,2,0.090909


Выборки в случаях A (доход более 1 000 000), D (доход от 30 000 до 50 000) и E (доход менее 30 000) довольно маленькие, потому их сложно считать репрезентативными. Если их не учитывать, то кредиты чаще вовзращают в срок те, чей доход выше.

### Зависимость между целью кредита и его возвратом в срок

In [63]:
create_debt_ratio_table('purpose_category')

,total,debt,ratio
purpose_category,,,
операции с автомобилем,4306,403,0.093590
операции с недвижимостью,10811,782,0.072334
получение образования,4013,370,0.092200
проведение свадьбы,2324,186,0.080034


Чаще всего люди возвращают кредит в случае, если берут его для свадьбы или недвижимости, и реже, если для учебы или автомобиля.

## Общий вывод:

Определенная зависимость между целями кредита/количеством детей/семейным положением/уровнем дохода и возвратом кредита в срок наблюдается, хотя нельзя сказать, что разница огромная - во всех случаях доля невозвращенных кредитов составляет от 7% до 10%. Тем не менее, эти 2-3% при больших количествах кредитов могут оказать значительное влияние на кредитный портфель банка. Так, кредиты чаще возвращают в срок клиенты без детей, но состоящие в браке, с высоким доходом.